In [1]:
import pandas as pd
import numpy as np

df = pd.read_pickle("../data/interim/rating.pkl")
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,28833,B01G15TGCU,1.0,1491601677000
0,0,29361,B01HRPJQ2S,5.0,1475448112000
0,0,32319,B06XQYN77L,1.0,1658498138614
0,0,37990,B07F9SG3RX,5.0,1673250487226
0,0,40014,B07L4GYFK9,5.0,1677469171342
...,...,...,...,...,...
999,999,69440,B0BYSG291N,5.0,1559594592832
999,999,70383,B0C37XY2JZ,5.0,1540929869776
999,999,71229,B0C662M3GG,5.0,1547336147049
999,999,71551,B0C78KPQYH,5.0,1552672042595


In [2]:
LEN_ITEM = len(df["item_id"].unique())

In [3]:
df = df.explode(["user_id", "rating", "timestamp"])
df

,user_id,item_id,parent_asin,rating,timestamp
0,0,28833,B01G15TGCU,1.0,1491601677000
0,0,29361,B01HRPJQ2S,5.0,1475448112000
0,0,32319,B06XQYN77L,1.0,1658498138614
0,0,37990,B07F9SG3RX,5.0,1673250487226
0,0,40014,B07L4GYFK9,5.0,1677469171342
...,...,...,...,...,...
999,999,69440,B0BYSG291N,5.0,1559594592832
999,999,70383,B0C37XY2JZ,5.0,1540929869776
999,999,71229,B0C662M3GG,5.0,1547336147049
999,999,71551,B0C78KPQYH,5.0,1552672042595


In [4]:
df = df.groupby("user_id").agg({
    "item_id": lambda x: x.tolist(),
    "rating": lambda x: x.tolist(),
}).reset_index(drop=False)

df["item_len"] = df["item_id"].apply(lambda x: len(x))

df

,user_id,item_id,rating,item_len
0,0,"[28833, 29361, 32319, 37990, 40014, 40697, 416...","[1.0, 5.0, 1.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",123
1,1,"[151, 177, 702, 1273, 1738, 2511, 2783, 2930, ...","[4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 5.0, ...",129
2,2,"[2532, 14867, 15799, 21812, 22038, 22315, 2236...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",124
3,3,"[218, 222, 2797, 2859, 2876, 3957, 5195, 5199,...","[5.0, 4.0, 3.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, ...",142
4,4,"[28, 31, 895, 1390, 1546, 1779, 1855, 1873, 21...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 2.0, 5.0, 4.0, ...",118
...,...,...,...,...
995,995,"[32306, 34442, 38700, 39334, 41441, 42668, 441...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, ...",148
996,996,"[11220, 12897, 13711, 14018, 14467, 15824, 188...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",169
997,997,"[25820, 28549, 28799, 29306, 29927, 30490, 326...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",118
998,998,"[12095, 13994, 18046, 18847, 23361, 25818, 324...","[5.0, 4.0, 3.0, 5.0, 4.0, 1.0, 5.0, 5.0, 3.0, ...",158


In [5]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy

total_item_id = np.arange(LEN_ITEM)

for idx, row in tqdm(df.iterrows()):

    neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    while(np.intersect1d(neg_item_idx, row["item_id"]).shape[0] > 0):
        neg_item_idx = np.random.choice(total_item_id, size=len(row["item_id"]), replace=False)
    # neg_item_idx = total_item_id[~np.isin(total_item_id, row["item_id"])]
    # neg_sampled_idx = np.random.choice([i for i in range(len(neg_item_idx))], size=len(row["item_id"]), replace=False)

    item_id = row["item_id"] + neg_item_idx.tolist()
    rating = row["rating"] + [0]*len(neg_item_idx)

    df.at[idx, "item_id"] = item_id
    df.at[idx, "rating"] = rating

df

1000it [00:34, 29.35it/s]


,user_id,item_id,rating,item_len
0,0,"[28833, 29361, 32319, 37990, 40014, 40697, 416...","[1.0, 5.0, 1.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, ...",123
1,1,"[151, 177, 702, 1273, 1738, 2511, 2783, 2930, ...","[4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 5.0, 4.0, 5.0, ...",129
2,2,"[2532, 14867, 15799, 21812, 22038, 22315, 2236...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",124
3,3,"[218, 222, 2797, 2859, 2876, 3957, 5195, 5199,...","[5.0, 4.0, 3.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, ...",142
4,4,"[28, 31, 895, 1390, 1546, 1779, 1855, 1873, 21...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 2.0, 5.0, 4.0, ...",118
...,...,...,...,...
995,995,"[32306, 34442, 38700, 39334, 41441, 42668, 441...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, ...",148
996,996,"[11220, 12897, 13711, 14018, 14467, 15824, 188...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",169
997,997,"[25820, 28549, 28799, 29306, 29927, 30490, 326...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",118
998,998,"[12095, 13994, 18046, 18847, 23361, 25818, 324...","[5.0, 4.0, 3.0, 5.0, 4.0, 1.0, 5.0, 5.0, 3.0, ...",158


In [6]:
df = df.explode(["item_id", "rating"])
df

,user_id,item_id,rating,item_len
0,0,28833,1.0,123
0,0,29361,5.0,123
0,0,32319,1.0,123
0,0,37990,5.0,123
0,0,40014,5.0,123
...,...,...,...,...
999,999,44930,0,193
999,999,14950,0,193
999,999,50080,0,193
999,999,22395,0,193


In [7]:
df.to_pickle("../data/processed/rating_engage.pkl")